## Overview
Welcome to the ARM/ASR Open Science Workshop Tutorial on the **Atmospheric data Community Toolkit (ACT)**
In this tutorial, you will learn some of the basic features of ACT with a focus on how you can use it to better utilize ARM's data quality information.  This will include using ARM's embedded quality control information that's included in many ARM NetCDF files and Data Quality Reports (DQR) which can be accessed through a webservice.

ACT is built around the xarray data object, which you can learn about more in the xarray tutorial or through information from __[Project Pythia](https://foundations.projectpythia.org/core/xarray.html)__. 

First we are going to start by importing all the necessary python libraries that we need

In [ ]:
import act
import matplotlib.pyplot as plt

## Download ARM Data
Next we are going to download the data we are going to use for this session using the __[ARM Live Data webservice](https://adc.arm.gov/armlive/)__.  

Since you have an ARM user account, you should be able to utilize this webservice as well.  All you need to do is login to get your token.  In case you are not able to get that information, we have a username and token set up just for training sessions.

In [ ]:
# Set your own username and token if you have it
username = 'armlive_training'
token = '4d26d88f91a2396f'

# ACT module for downloading data from the ARM web service
results = act.discovery.download_data(username, token, 'sgpmfrsr7nchE11.b1', '2021-03-29', '2021-03-29')
print(results)

### Reading in a NetCDF File
Congratulations, you just downloaded a file from just the command line!  Next up is to read the file into an xarray object using the ACT reader.  We then can use Jupyter to print out an interactive listing of everything in the object.

In [ ]:
obj = act.io.armfiles.read_netcdf(results)
obj

### Clean up the object to CF Standards
In order to utilize all the ACT QC modules, we need to clean up the object to follow __[Climate and Forecast (CF) standards](https://cfconventions.org/)__.  

In [ ]:
obj.clean.cleanup()

### Next, let's plot up a variable and it's QC to see what we're working with
For this example, we'll use diffuse_hemisp_narrowband_filter4

In [ ]:
variable = 'diffuse_hemisp_narrowband_filter4'

# Create a plotting display object with 2 plots
display = act.plotting.TimeSeriesDisplay(obj, figsize=(15, 10), subplot_shape=(2,))

# Plot up the diffuse variable in the first plot
display.plot(variable, subplot_index=(0,))

# Plot up a day/night background
display.day_night_background(subplot_index=(0,))

# Plot up the QC variable in the second plot
display.qc_flag_block_plot(variable, subplot_index=(1,))

In [ ]:
# Now lets remove some of these outliers
obj.qcfilter.datafilter(variable, rm_tests=[2, 3], del_qc_var=False)

# And plot the data again
# Create a plotting display object with 2 plots
display = act.plotting.TimeSeriesDisplay(obj, figsize=(15, 10), subplot_shape=(2,))

# Plot up the diffuse variable in the first plot
display.plot(variable, subplot_index=(0,))

# Plot up a day/night background
display.day_night_background(subplot_index=(0,))

# Plot up the QC variable in the second plot
display.qc_flag_block_plot(variable, subplot_index=(1,))

### Query the DQR Webservice
Since the embedded QC is not removing all the outliers, let's check to see if there are any DQRs using ARMs DQR Webservice.  The great thing is, that ACT has codes for working with this webservice.

In [ ]:
# Query the ARM DQR Webservice
obj = act.qc.arm.add_dqr_to_qc(obj, variable=variable)

#And plot again!
# Create a plotting display object with 2 plots
display = act.plotting.TimeSeriesDisplay(obj, figsize=(15, 10), subplot_shape=(2,))

# Plot up the diffuse variable in the first plot
display.plot(variable, subplot_index=(0,))

# Plot up a day/night background
display.day_night_background(subplot_index=(0,))

# Plot up the QC variable in the second plot
display.qc_flag_block_plot(variable, subplot_index=(1,))

### Add QC Tests
ACT has a number of additional __[QC tests](https://arm-doe.github.io/ACT/API/generated/act.qc.QCTests.html#act.qc.QCTests)__ that could be applied to the data.  For this next example, let's apply a new maximum test.

In [ ]:
# Add a new maximum tests
obj.qcfilter.add_greater_test(variable, 0.4, test_meaning='New maximum tests limit')

# Filter that test out
obj.qcfilter.datafilter(variable, rm_tests=[5], del_qc_var=False)

#And plot again!
# Create a plotting display object with 2 plots
display = act.plotting.TimeSeriesDisplay(obj, figsize=(15, 10), subplot_shape=(2,))

# Plot up the diffuse variable in the first plot
display.plot(variable, subplot_index=(0,))

# Plot up a day/night background
display.day_night_background(subplot_index=(0,))

# Plot up the QC variable in the second plot
display.qc_flag_block_plot(variable, subplot_index=(1,))

### Instrument Specific QC Tests
ACT has a growing library of instrument specific tests such as the __[fast-fourier transform test](https://arm-doe.github.io/ACT/API/generated/act.qc.fft_shading_test.html#act.qc.fft_shading_test)__ to detect shading which was adapted from Alexandrov et al 2007.  The adaption is that it is applied in a moving window style approach.  **Note - Check out the webpage as an example of how we are including references to papers behind the codes**

Let's apply it and see how it compares with the DQR!

In [ ]:
# Apply test
obj = act.qc.fft_shading_test(obj, variable=variable)

# Create a plotting display object with 2 plots
display = act.plotting.TimeSeriesDisplay(obj, figsize=(15, 10), subplot_shape=(2,))

# Plot up the diffuse variable in the first plot
display.plot(variable, subplot_index=(0,))

# Plot up a day/night background
display.day_night_background(subplot_index=(0,))

# Plot up the QC variable in the second plot
display.qc_flag_block_plot(variable, subplot_index=(1,))

### Conclusion
In this tutorial, we have shown you how to download data from ARM's Data Live web service, visualize the data and QC information, query the DQR webservice, filter data based on the QC, and add new QC tests to the dataset.  After all this work, you can easily save the xarray object to a NetCDF file using `obj.to_netcdf('filename.nc')` and all that data will be saved and usable in Python and ACT.

Please checkout the __[ACT Github repository](https://github.com/ARM-DOE/ACT)__ for the latest and greatest information, including our documentation which has __[examples](https://arm-doe.github.io/ACT/source/auto_examples/index.html)__ that can be downloading in python or Jupyter Notebook formats.